In [1]:
import torch
import torchvision
import numpy as np
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
from skimage import io, transform
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# load image
mnist = input_data.read_data_sets('../../MNIST', one_hot=True)
z_dim = 10
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 128
cnt = 0
num_epochs = 300
batch_size = 128
learning_rate = 1e-3
print X_dim

In [2]:
if not os.path.exists('./results/conv/'):
    os.mkdir('./results/conv/')


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 3, 360, 270)
    return x

img_transform = transforms.Compose([
    transforms.ToTensor(),
    ])


In [4]:
#define model
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=3, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            #denn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=3),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 2, stride=2, padding=0),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 3, 3, stride=3, padding=0),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

In [5]:
# train network
for it in range(10000):
    img, _ = mnist.train.next_batch(batch_size)
    img = Variable(torch.from_numpy(img)).cuda()
    # ===================forward=====================
    output = model(img)
    #print output
    loss = criterion(output, img)
    # print loss
    #input()
    # ===================backward====================
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # ===================log========================
    
    if it % batch_size == 0:
        epoch = it/batch_size
        print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0]))
        pic = to_img(output.cpu().data)
        save_image(pic, './results/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './garment_autoencoder.pth')

epoch [1/300], loss:0.6089
epoch [2/300], loss:0.0496
epoch [3/300], loss:0.0430
epoch [4/300], loss:0.0418
epoch [5/300], loss:0.0308
epoch [6/300], loss:0.0297
epoch [7/300], loss:0.0255
epoch [8/300], loss:0.0201
epoch [9/300], loss:0.0227
epoch [10/300], loss:0.0208
epoch [11/300], loss:0.0160
epoch [12/300], loss:0.0263
epoch [13/300], loss:0.0169
epoch [14/300], loss:0.0202
epoch [15/300], loss:0.0168
epoch [16/300], loss:0.0198
epoch [17/300], loss:0.0158
epoch [18/300], loss:0.0199
epoch [19/300], loss:0.0155
epoch [20/300], loss:0.0173
epoch [21/300], loss:0.0170
epoch [22/300], loss:0.0128
epoch [23/300], loss:0.0173
epoch [24/300], loss:0.0118
epoch [25/300], loss:0.0189
epoch [26/300], loss:0.0183
epoch [27/300], loss:0.0160
epoch [28/300], loss:0.0149
epoch [29/300], loss:0.0172
epoch [30/300], loss:0.0156
epoch [31/300], loss:0.0145
epoch [32/300], loss:0.0121
epoch [33/300], loss:0.0160
epoch [34/300], loss:0.0155
epoch [35/300], loss:0.0152
epoch [36/300], loss:0.0136
e

epoch [288/300], loss:0.0094
epoch [289/300], loss:0.0098
epoch [290/300], loss:0.0076
epoch [291/300], loss:0.0107
epoch [292/300], loss:0.0093
epoch [293/300], loss:0.0116
epoch [294/300], loss:0.0093
epoch [295/300], loss:0.0081
epoch [296/300], loss:0.0067
epoch [297/300], loss:0.0060
epoch [298/300], loss:0.0087
epoch [299/300], loss:0.0085
epoch [300/300], loss:0.0106
